In [ ]:
import psycopg2
from psycopg2.extras import execute_values
import pandas as pd
import os
import sys
import base64
from bs4 import BeautifulSoup
import re

In [ ]:
import tiktoken
import PyPDF2
import re

In [ ]:
# Caminho para o diretório onde o módulo conexoes.py está localizado
caminho_para_conexoes = '/Users/alexandreloureiro/bb/mefis/pln'

# Adicionando o caminho ao sys.path
if caminho_para_conexoes not in sys.path:
    sys.path.append(caminho_para_conexoes)

# Importando a função postgres do módulo conexoes
from conexoes import postgres

# Conectando ao banco de dados
conn, engine = postgres()
cursor = conn.cursor()


In [ ]:
import pdfplumber

# Caminho para o arquivo PDF
pdf_path = '/Users/alexandreloureiro/bb/mefis/estudo_risco.pdf'

# Caminho para o arquivo de saída
output_path = 'relatorio.txt'

# Abrir o arquivo PDF
with pdfplumber.open(pdf_path) as pdf:
    # Inicializar uma lista para armazenar o texto de cada página
    paginas_texto = []
    
    # Iterar por todas as páginas
    for page_num, page in enumerate(pdf.pages, start=1):
        print(f'Processando página {page_num} de {len(pdf.pages)}...')
        
        # Extrair o texto da página, mantendo o layout
        texto = page.extract_text(x_tolerance=1, y_tolerance=1)
        
        if texto:
            paginas_texto.append(texto)
        else:
            print(f'Aviso: Nenhum texto encontrado na página {page_num}.')

# Unir o texto de todas as páginas
texto_completo = '\n\n'.join(paginas_texto)

# Salvar o texto no arquivo 'relatorio.txt'
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(texto_completo)

print(f'\nExtração concluída. O texto foi salvo em "{output_path}".')



In [ ]:
import PyPDF2

# Caminho para o arquivo PDF
pdf_path = '/Users/alexandreloureiro/bb/mefis/estudo_risco.pdf'

# Caminho para o arquivo de saída
output_path = 'relatorio2.txt'

# Abrir o arquivo PDF em modo de leitura binária
with open(pdf_path, 'rb') as pdf_file:
    # Criar um objeto leitor do PDF
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    
    # Inicializar uma lista para armazenar o texto de cada página
    paginas_texto = []
    
    # Iterar por todas as páginas do PDF
    for page_num in range(len(pdf_reader.pages)):
        print(f'Processando página {page_num + 1} de {len(pdf_reader.pages)}...')
        # Obter a página atual
        page = pdf_reader.pages[page_num]
        # Extrair o texto da página
        texto = page.extract_text()
        if texto:
            paginas_texto.append(texto)
        else:
            print(f'Aviso: Nenhum texto encontrado na página {page_num + 1}.')

# Unir o texto de todas as páginas
texto_completo = '\n\n'.join(paginas_texto)

# Salvar o texto no arquivo 'relatorio2.txt'
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(texto_completo)

print(f'\nExtração concluída. O texto foi salvo em "{output_path}".')


In [ ]:
from pdfminer.high_level import extract_text
import re
import os


# Lista de caminhos para os arquivos PDF
pdf_files = [
    '/Users/alexandreloureiro/bb/mefis/estudo_risco.pdf',
    '/Users/alexandreloureiro/bb/mefis/estrategia.pdf',
    '/Users/alexandreloureiro/bb/mefis/notas/ecbb.pdf',
    '/Users/alexandreloureiro/bb/mefis/notas/mrld_5.pdf',
]

for pdf_path in pdf_files:
    # Verificar se o arquivo existe
    if not os.path.isfile(pdf_path):
        print(f'Arquivo não encontrado: {pdf_path}')
        continue

    # Obter o nome base do arquivo (sem o caminho completo)
    base_name = os.path.basename(pdf_path)
    # Remover a extensão '.pdf' do nome base
    file_name_no_ext = os.path.splitext(base_name)[0]
    # Criar o nome do arquivo de saída
    output_path = f'rel_temp_{file_name_no_ext}.txt'

    print(f'Processando o arquivo: {pdf_path}')
    # Extrair todo o texto do PDF
    texto_completo = extract_text(pdf_path)

    # Remover múltiplas quebras de linha consecutivas
    texto_limpo = re.sub(r'\n\s*\n+', '\n\n', texto_completo)

    # Remover múltiplos espaços dentro das linhas
    texto_limpo = re.sub(r'[ ]{2,}', ' ', texto_limpo)

    # Salvar o texto no arquivo de saída
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(texto_limpo)

    print(f'Extração concluída para "{pdf_path}". O texto foi salvo em "{output_path}".\n')



In [ ]:
from pdfminer.high_level import extract_text
import re
import os
import pandas as pd

# Lista de caminhos para os arquivos PDF
pdf_files = [
    '/Users/arquivo.pdf',
    '/Users/arquivo.pdf',
    '/Users/arquivo.pdf',
    '/Users/arquivo.pdf',
]

# Definir o modelo e obter o codificador apropriado
modelo = 'gpt-4'
try:
    encoding = tiktoken.encoding_for_model(modelo)
except KeyError:
    print(f'Modelo "{modelo}" não encontrado. Usando codificação padrão "cl100k_base".')
    encoding = tiktoken.get_encoding('cl100k_base')

# Lista para armazenar os dados
dados = []

for pdf_path in pdf_files:
    # Verificar se o arquivo existe
    if not os.path.isfile(pdf_path):
        print(f'Arquivo não encontrado: {pdf_path}')
        continue

    # Obter o nome base do arquivo (sem o caminho completo)
    base_name = os.path.basename(pdf_path)
    # Remover a extensão '.pdf' do nome base
    file_name_no_ext = os.path.splitext(base_name)[0]

    print(f'Processando o arquivo: {pdf_path}')
    # Extrair todo o texto do PDF
    texto_completo = extract_text(pdf_path)

    # Remover múltiplas quebras de linha consecutivas
    texto_limpo = re.sub(r'\n\s*\n+', '\n\n', texto_completo)

    # Remover múltiplos espaços dentro das linhas
    texto_limpo = re.sub(r'[ ]{2,}', ' ', texto_limpo)

    # Remover espaços em branco no início e no fim das linhas
    linhas = texto_limpo.split('\n')
    linhas = [linha.strip() for linha in linhas]

    # Remover linhas completamente vazias
    linhas = [linha for linha in linhas if linha != '']

    # Reunir as linhas em um único texto
    texto_final = '\n'.join(linhas)

    # Calcular o número de tokens usando tiktoken
    tokens = encoding.encode(texto_final)
    numero_de_tokens = len(tokens)

    # Adicionar os dados à lista
    dados.append({
        'path': pdf_path,
        'filename': base_name,
        'extracted_text': texto_final,
        'numero_de_tokens': numero_de_tokens
    })

    print(f'Extração concluída para "{pdf_path}". Número de tokens: {numero_de_tokens}\n')

# Criar o dataframe
df = pd.DataFrame(dados, columns=['path', 'filename', 'extracted_text', 'numero_de_tokens'])

# Exibir o dataframe
print(df)

# Opcional: Salvar o dataframe em um arquivo CSV ou Excel
# df.to_csv('textos_extraidos_com_tokens.csv', index=False)
# df.to_excel('textos_extraidos_com_tokens.xlsx', index=False)





In [ ]:
df

In [ ]:
modelo = 'gpt-4'
encoding = tiktoken.encoding_for_model(modelo)

In [ ]:
tokens = encoding.encode(texto_limpo)

In [ ]:
c

In [ ]:
print(f'O número de tokens para o modelo {modelo} é: {numero_de_tokens}')

In [ ]:
from pdfminer.high_level import extract_text
from pdfminer.pdfparser import PDFSyntaxError
import re
import os
from datetime import datetime

def limpar_texto(texto):
    """
    Função para limpar e formatar o texto extraído do PDF
    """
    # Remove caracteres especiais e formata o texto
    texto = re.sub(r'\f', '', texto)  # Remove form feeds
    texto = re.sub(r'[ \t]+', ' ', texto)  # Remove múltiplos espaços e tabs
    texto = re.sub(r'\n\s*\n+', '\n\n', texto)  # Remove múltiplas quebras de linha
    texto = re.sub(r'_{3,}', '_____', texto)  # Padroniza linhas de underscore
    texto = re.sub(r'\.{3,}', '...', texto)  # Padroniza reticências
    
    # Remove espaços antes de pontuação
    texto = re.sub(r'\s+([.,;!?])', r'\1', texto)
    
    # Remove linhas que contêm apenas números de página
    texto = re.sub(r'^\s*\d+\s*$', '', texto, flags=re.MULTILINE)
    
    # Remove espaços no início e fim de cada linha
    linhas = [linha.strip() for linha in texto.split('\n')]
    texto = '\n'.join(linhas)
    
    # Remove linhas vazias consecutivas mantendo no máximo uma
    texto = re.sub(r'\n{3,}', '\n\n', texto)
    
    return texto.strip()

def extrair_texto_pdf(caminho_pdf, caminho_saida):
    """
    Extrai e processa texto de um arquivo PDF
    """
    try:
        # Verifica se o arquivo existe
        if not os.path.exists(caminho_pdf):
            raise FileNotFoundError(f"O arquivo '{caminho_pdf}' não foi encontrado.")
            
        # Cria backup do arquivo de saída se ele já existir
        if os.path.exists(caminho_saida):
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            backup_path = f"{caminho_saida}.{timestamp}.bak"
            os.rename(caminho_saida, backup_path)
            print(f"Backup do arquivo anterior criado: {backup_path}")
        
        print("Extraindo texto do PDF...")
        texto_completo = extract_text(caminho_pdf)
        
        print("Limpando e formatando o texto...")
        texto_limpo = limpar_texto(texto_completo)
        
        # Adiciona informações do processamento
        cabecalho = f"""# Texto extraído do arquivo: {os.path.basename(caminho_pdf)}
# Data de extração: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
# Processado com pdfminer.six
{'='*80}\n\n"""
        
        texto_final = cabecalho + texto_limpo
        
        # Salva o texto processado
        with open(caminho_saida, 'w', encoding='utf-8') as f:
            f.write(texto_final)
            
        # Estatísticas do processamento
        total_caracteres = len(texto_final)
        total_palavras = len(texto_final.split())
        total_linhas = len(texto_final.splitlines())
        
        print(f"\nProcessamento concluído com sucesso!")
        print(f"Arquivo salvo em: {caminho_saida}")
        print(f"\nEstatísticas:")
        print(f"- Total de caracteres: {total_caracteres:,}")
        print(f"- Total de palavras: {total_palavras:,}")
        print(f"- Total de linhas: {total_linhas:,}")
        
    except PDFSyntaxError:
        print("Erro: O arquivo PDF está corrompido ou não pode ser lido.")
    except PermissionError:
        print(f"Erro: Sem permissão para acessar ou criar o arquivo '{caminho_saida}'")
    except Exception as e:
        print(f"Erro inesperado: {str(e)}")

# Caminhos dos arquivos
pdf_path = '/Users/alexandreloureiro/bb/mefis/estudo_risco.pdf'
output_path = 'relatorio3.txt'

# Executa a extração
extrair_texto_pdf(pdf_path, output_path)

In [ ]:
# Importando as bibliotecas necessárias
import PyPDF2
import re

def limpar_texto(texto):
    """
    Limpa o texto removendo linhas em branco extras e espaços desnecessários.
    
    Args:
        texto (str): Texto a ser limpo
        
    Returns:
        str: Texto limpo
    """
    # Remove caracteres especiais e espaços extras
    texto = re.sub(r'_+', '_', texto)  # Substitui múltiplos underscores por um único
    texto = re.sub(r'\s*\n\s*\n\s*', '\n\n', texto)  # Substitui múltiplas linhas em branco por uma única
    texto = re.sub(r'\s+', ' ', texto)  # Substitui múltiplos espaços por um único
    texto = texto.strip()  # Remove espaços no início e fim
    
    # Divide o texto em linhas
    linhas = texto.split('\n')
    
    # Remove linhas que só contêm espaços ou caracteres especiais
    linhas = [linha.strip() for linha in linhas if linha.strip() and not linha.strip().startswith('_' * 10)]
    
    # Junta as linhas de volta com uma quebra de linha dupla
    return '\n\n'.join(linhas)

def extrair_texto_pdf(caminho_pdf):
    """
    Extrai texto de um arquivo PDF e limpa formatação extra.
    
    Args:
        caminho_pdf (str): Caminho para o arquivo PDF
        
    Returns:
        str: Texto extraído e limpo do PDF
    """
    with open(caminho_pdf, 'rb') as arquivo:
        leitor_pdf = PyPDF2.PdfReader(arquivo)
        texto_completo = ""
        
        print(f"Total de páginas no PDF: {len(leitor_pdf.pages)}")
        
        for num_pagina, pagina in enumerate(leitor_pdf.pages, 1):
            # Extrai o texto da página
            texto_pagina = pagina.extract_text()
            
            # Limpa o texto da página
            texto_pagina_limpo = limpar_texto(texto_pagina)
            
            # Adiciona cabeçalho da página e o texto limpo
            texto_completo += f"\n=== Página {num_pagina} ===\n\n"
            texto_completo += texto_pagina_limpo + "\n"
        
        # Limpa novamente o texto completo para garantir consistência
        texto_completo = limpar_texto(texto_completo)
        return texto_completo





In [ ]:
# Extrair e limpar o texto
texto_extraido = extrair_texto_pdf(caminho_do_pdf)

# Mostrar uma amostra do texto limpo
print("\nAmostra do texto limpo:")
print(texto_extraido[:500])

# Salvar o texto limpo em um arquivo
with open("texto_extraido_limpo.txt", "w", encoding="utf-8") as arquivo_texto:
    arquivo_texto.write(texto_extraido)

# Para visualizar de forma mais organizada no Jupyter
from IPython.display import display, Markdown
display(Markdown(texto_extraido))

In [ ]:
# Lendo os dados da tabela
query = """
    SELECT cd_atvd, cd_rel_atvd, cd_tip_rel_atvd, cd_tip_est_rel, tx_rel_atvd,
           cd_usu_rsp_rel, cd_uor_rsp_rel, ts_vld_rel, db2_generated_rowid_for_lobs,
           nr_trabalho, tx_constatacao, rel_hash, rel_s_hash
    FROM ptai.com_resultado_analise;
"""

df = pd.read_sql(query, engine)

In [ ]:
df.shape

In [ ]:
# Verifica se a coluna 'tx_rel_atvd_sem_html' existe; caso contrário, cria
cursor.execute("""
    SELECT column_name FROM information_schema.columns
    WHERE table_schema = 'ptai' AND table_name = 'com_resultado_analise' AND column_name = 'tx_rel_atvd_sem_html';
""")
column_exists = cursor.fetchone()

if not column_exists:
    cursor.execute("""
        ALTER TABLE ptai.com_resultado_analise ADD COLUMN tx_rel_atvd_sem_html TEXT;
    """)
    conn.commit()

cursor.execute("""
    CREATE TABLE IF NOT EXISTS ptai.com_resultado_analise_imagens (
        nr_trabalho INTEGER,
        image_number INTEGER,
        image_data BYTEA,
        image_type TEXT,
        path_imagem TEXT
    );
""")
conn.commit()

In [ ]:
def process_row(row):
    html_content = row['tx_rel_atvd']
    nr_trabalho = row['nr_trabalho']

    # Verifica se o conteúdo HTML é válido
    if not isinstance(html_content, str):
        return '', []

    # Utiliza o BeautifulSoup para analisar o conteúdo HTML
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extrai o texto puro, removendo todas as tags HTML
    text_content = soup.get_text(separator='\n')

    # Contador para nomear as imagens na ordem em que aparecem
    image_counter = 1

    images_list = []

    # Encontra todas as tags <img> no conteúdo HTML
    for img_tag in soup.find_all('img'):
        img_src = img_tag.get('src', '')
        # Verifica se o 'src' é um data URL com imagem em base64
        data_url_match = re.match(r'data:image/(\w+);base64,(.*)', img_src)
        if data_url_match:
            img_type = data_url_match.group(1)  # Obtém o tipo da imagem (jpeg, png, etc.)
            base64_data = data_url_match.group(2)  # Obtém os dados base64
            # Decodifica os dados base64 para bytes de imagem
            try:
                img_data = base64.b64decode(base64_data)
            except Exception as e:
                print(f"Erro ao decodificar imagem base64 para nr_trabalho {nr_trabalho}: {e}")
                continue
            # Define o nome e o caminho do arquivo de imagem
            image_filename = f"{nr_trabalho}_{image_counter}.{img_type}"
            image_path = os.path.join('imagem_relatorio', image_filename)
            # Salva a imagem no diretório especificado
            with open(image_path, 'wb') as f:
                f.write(img_data)
            # Armazena os dados da imagem na lista, incluindo o caminho da imagem
            images_list.append((nr_trabalho, image_counter, psycopg2.Binary(img_data), img_type, image_path))
            # Incrementa o contador de imagens
            image_counter += 1

    # Retorna o texto puro e a lista de imagens
    return text_content, images_list

In [ ]:
# Verifica se a pasta 'imagem_relatorio' existe; caso contrário, cria
if not os.path.exists('imagem_relatorio'):
    os.makedirs('imagem_relatorio')

In [ ]:
# Aplica a função a cada linha do DataFrame para criar a nova coluna 'tx_rel_atvd_sem_html' e coletar as imagens
result = df.apply(process_row, axis=1, result_type='expand')
df['tx_rel_atvd_sem_html'] = result[0]
df['images'] = result[1]

In [ ]:
# Coleta todas as imagens para inserir no banco de dados
images_to_insert = []
for images_list in df['images']:
    images_to_insert.extend(images_list)

In [ ]:
# Insere as imagens no banco de dados
if images_to_insert:
    sql_insert_images = """
        INSERT INTO ptai.com_resultado_analise_imagens (nr_trabalho, image_number, image_data, image_type, path_imagem)
        VALUES (%s, %s, %s, %s, %s);
    """
    cursor.executemany(sql_insert_images, images_to_insert)
    conn.commit()

In [ ]:
# Remove a coluna 'images' do DataFrame
df.drop('images', axis=1, inplace=True)

# Prepara os dados para atualizar a tabela original
data_to_update = list(zip(df['tx_rel_atvd_sem_html'], df['cd_atvd']))

In [ ]:
# Atualiza a tabela original com o novo texto
sql_update = """
    UPDATE ptai.com_resultado_analise
    SET tx_rel_atvd_sem_html = %s
    WHERE cd_atvd = %s;
"""

cursor.executemany(sql_update, data_to_update)
conn.commit()

In [ ]:
# Fecha o cursor e a conexão
cursor.close()
conn.close()

In [ ]:
# Reabrir a conexão ao banco de dados
conn, engine = postgres()
cursor = conn.cursor()

# Consulta para recuperar as imagens
query_images = """
    SELECT nr_trabalho, image_number, image_data, image_type
    FROM ptai.com_resultado_analise_imagens;
"""

df_images = pd.read_sql(query_images, conn)


In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO

# Função para exibir uma imagem
def display_image(row):
    img_data = row['image_data']
    img_type = row['image_type']
    # Converter os dados binários em uma imagem PIL
    image = Image.open(BytesIO(img_data))
    # Exibir a imagem
    plt.imshow(image)
    plt.axis('off')
    plt.title(f"nr_trabalho: {row['nr_trabalho']}, image_number: {row['image_number']}")
    plt.show()

# Exibir todas as imagens
for index, row in df_images.iterrows():
    display_image(row)
